In [49]:
from utils.BackTest import Engine, Strategy
import pandas as pd
import xgboost as xgb
import numpy as np
from torch.utils.data import Dataset,DataLoader

In [50]:
df = pd.read_csv(r'../data/processed/merged_df_with_signals.csv')

In [51]:
df

,datetime,reddit_smoothed_sentiment_weibull_week,news_smoothed_sentiment_weibull_week,tele_smoothed_sentiment_weibull_week,Open,High,Low,Close,TOTAL_TRADES,TOTAL_TRADES_BUY,...,reddit_sentiment_log,news_sentiment_log,tele_sentiment_log,reddit_Signal,news_Signal,tele_Signal,typical_price,DATE,VWAP,VWAP_Signal
0,2024-01-27 13:00:00,0.000000,-857.923901,6.872941,41750.00,41806.55,41725.21,41742.44,35569,10171,...,0.000000,-6.755482,2.060767,0.0,-1.0,1.0,41758.066667,2024-01-27,41758.066667,-1
1,2024-01-27 14:00:00,0.027001,-867.694664,6.872941,41742.44,41818.39,41731.01,41766.00,27238,12281,...,0.026643,-6.766796,2.060767,1.0,-1.0,1.0,41771.800000,2024-01-27,41765.490372,1
2,2024-01-27 15:00:00,0.164191,-875.992357,6.872941,41766.00,41894.87,41733.60,41814.00,32410,14004,...,0.152026,-6.776304,2.060767,1.0,-1.0,1.0,41814.156667,2024-01-27,41784.423590,1
3,2024-01-27 16:00:00,0.300527,-882.407146,6.892280,41814.00,41895.74,41791.22,41810.18,30278,13664,...,0.262769,-6.783594,2.063227,1.0,-1.0,1.0,41832.380000,2024-01-27,41798.313570,1
4,2024-01-27 17:00:00,0.435303,-886.430380,6.892280,41810.18,41845.34,41774.07,41779.28,24681,9330,...,0.361376,-6.788139,2.063227,1.0,-1.0,1.0,41799.563333,2024-01-27,41798.533122,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10171,2025-03-26 08:00:00,-0.487101,-43.013907,12.500145,88140.45,88200.00,87886.10,87959.73,59816,30655,...,-0.396829,-3.780636,2.601147,-1.0,-1.0,1.0,88015.276667,2025-03-26,87642.510178,1
10172,2025-03-26 09:00:00,-0.456887,-39.331997,12.500145,87959.73,88056.61,87837.49,87939.99,43067,23857,...,-0.376302,-3.692921,2.601147,-1.0,-1.0,1.0,87944.696667,2025-03-26,87661.828105,1
10173,2025-03-26 10:00:00,-0.427685,-37.038648,12.500145,87939.99,88259.81,87939.99,88208.47,55584,29793,...,-0.356054,-3.634124,2.601147,-1.0,-1.0,1.0,88136.090000,2025-03-26,87701.167882,1
10174,2025-03-26 11:00:00,-0.399548,-37.530942,12.500145,88208.47,88242.00,87923.91,87954.96,67105,30529,...,-0.336149,-3.647040,2.601147,-1.0,-1.0,1.0,88040.290000,2025-03-26,87742.875495,1


In [52]:
base_price_data = ['datetime','Close','Open','High','Low','VOLUME']

technical_indicators = [
    # Trend Indicators
    'EMA_8', 'EMA_13', 'EMA_21', 'EMA_Signal', 'EMA_short', 'EMA_long',
    'SMA20', 'SMA_50', 'SMA_200', 'GoldenCross_Signal',

    # Momentum Indicators
    'MACD', 'Signal_Line', 'MACD_Signal', 'MACD_Hist',
    'RSI_14', 'BB_RSI_Signal', 'RSI_Signal',
    '%K', '%D', 'Stochastic_Signal',

    # Volatility Indicators
    'STD20', 'Upper_BB', 'Lower_BB', 'ATR', 'TR',

    # Volume-Based Indicators
    'OBV', 'OBV_EMA20', 'OBV_Signal', 'VWAP_Signal'
]

sentiment_indicators = [
    'reddit_sentiment_log',
    'tele_sentiment_log',
    'news_sentiment_log',
]

# Create a new DataFrame with the selected columns
selected_columns = base_price_data + technical_indicators + sentiment_indicators
df_selected = df[selected_columns]

In [53]:
working_frame = df_selected.copy().set_index(pd.to_datetime(df_selected['datetime'])).drop(columns=['datetime'])
working_frame

,Close,Open,High,Low,VOLUME,EMA_8,EMA_13,EMA_21,EMA_Signal,EMA_short,...,Lower_BB,ATR,TR,OBV,OBV_EMA20,OBV_Signal,VWAP_Signal,reddit_sentiment_log,tele_sentiment_log,news_sentiment_log
datetime,,,,,,,,,,,,,,,,,,,,,
2024-01-27 13:00:00,41742.44,41750.00,41806.55,41725.21,419.38468,41729.550796,41714.525206,41617.761701,1,41720.541302,...,41524.488454,175.710000,81.34,-419.38468,-65.712408,0,-1,0.000000,2.060767,-6.755482
2024-01-27 14:00:00,41766.00,41742.44,41818.39,41731.01,493.43458,41737.650619,41721.878748,41631.237910,1,41727.534948,...,41541.267296,140.500000,87.38,493.43458,-12.460314,1,1,0.026643,2.060767,-6.766796
2024-01-27 15:00:00,41814.00,41766.00,41894.87,41733.60,581.25856,41754.617148,41735.038927,41647.852645,1,41740.837263,...,41561.147977,109.318333,161.27,581.25856,44.084341,0,1,0.152026,2.060767,-6.776304
2024-01-27 16:00:00,41810.18,41814.00,41895.74,41791.22,609.18361,41766.964449,41745.773366,41662.609678,1,41751.505377,...,41571.210298,101.921667,104.52,-609.18361,-18.131655,-1,1,0.262769,2.063227,-6.783594
2024-01-27 17:00:00,41779.28,41810.18,41845.34,41774.07,448.23397,41769.701238,41750.560028,41673.216071,1,41755.778396,...,41588.600128,99.801667,71.27,-448.23397,-59.093780,0,-1,0.361376,2.063227,-6.788139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-26 08:00:00,87959.73,88140.45,88200.00,87886.10,525.11331,87756.884904,87689.841782,87613.211597,1,87700.669674,...,87158.682498,418.066667,313.90,-525.11331,59.670458,-1,1,-0.396829,2.601147,-3.780636
2025-03-26 09:00:00,87939.99,87959.73,88056.61,87837.49,393.09750,87797.574925,87725.577241,87642.918724,1,87737.488186,...,87170.539414,392.118333,219.12,-393.09750,16.549700,0,1,-0.376302,2.601147,-3.692921
2025-03-26 10:00:00,88208.47,87939.99,88259.81,87939.99,556.20541,87888.884942,87794.561921,87694.332477,1,87809.946926,...,87154.234720,378.148333,319.82,556.20541,67.945482,1,1,-0.356054,2.601147,-3.634124


In [54]:
loaded_model = xgb.XGBRegressor()
loaded_model.load_model('xgboost_model.json')

In [59]:
class XGBoostStrategy(Strategy):
    def __init__(self,model,initial_data=None):
        super().__init__()
        self.model = model 
        self.initial_data = initial_data

    def on_bar(self):

        temp_df = pd.concat([self.initial_data, self.data])

        features = self.sequencing_data(temp_df, window_size=7)

        predictions = self.model.predict(features)
        signal = self.determine_signal(predictions)
        if signal == 1:
            self.buy(
                "btc",
               size = round(0.10 * (self.cash) / self.close(), 8)
            )
        elif signal == -1:
            self.sell(
                "btc",
                size = round(0.10 * (self.cash) / self.close(), 8)
            )
        else:
            pass

    def determine_signal(self, predictions):
        """Determine buy/sell/hold signal based on predictions."""
        future_forecasts = predictions.flatten().tolist()  # Convert to list if needed
        close = self.close()

        above_close = sum(forecast > close for forecast in future_forecasts)
        below_close = sum(forecast < close for forecast in future_forecasts)

        if above_close >= 3:
            return 1  # Buy
        elif below_close >= 3:
            return -1  # Sell
        else:
           return 0  # Hold
        
    def sequencing_data(self, df, window_size=7):
        start_time = self.current_idx - pd.Timedelta(hours=window_size)
        past_data = df.loc[start_time:self.current_idx - pd.Timedelta(hours=1)]

        # Collect past feature lists
        past_features = [
            past_data['High'].tolist(),
            past_data['Low'].tolist(),
            past_data['Open'].tolist(),
            past_data['VOLUME'].tolist()
        ]

        # Technical Indicators
        indicators = [
            'EMA_8', 'EMA_13', 'EMA_21', 'EMA_Signal', 'EMA_short', 'EMA_long',
            'SMA20', 'SMA_50', 'SMA_200', 'GoldenCross_Signal',
            'MACD', 'Signal_Line', 'MACD_Signal', 'MACD_Hist',
            'RSI_14', 'BB_RSI_Signal', 'RSI_Signal',
            '%K', '%D', 'Stochastic_Signal',
            'STD20', 'Upper_BB', 'Lower_BB', 'ATR', 'TR',
            'OBV', 'OBV_EMA20', 'OBV_Signal', 'VWAP_Signal'
        ]
        
        for indicator in indicators:
            past_features.append(past_data[indicator].tolist())

        # Sentiment indicators
        sentiments = ['reddit_sentiment_log', 'tele_sentiment_log', 'news_sentiment_log']
        
        for sentiment in sentiments:
            past_features.append(past_data[sentiment].tolist())

        # Convert to 1D NumPy array and reshape to (1, 252)
        past_features_array = np.concatenate(past_features).reshape(1, -1)
        
        return past_features_array  # Shape: (1, 252)

In [69]:
initial_val_frame = working_frame[(working_frame.index >= pd.to_datetime('2025-01-01') - pd.Timedelta(hours=7)) & (working_frame.index < '2025-01-01')]
validate_frame = working_frame[working_frame.index >= '2025-01-01']
initial_test_frame = working_frame[(working_frame.index >= pd.to_datetime('2025-03-01') - pd.Timedelta(hours=7)) & (working_frame.index < '2025-03-01')]
test_frame = working_frame[working_frame.index >= '2025-03-01']

In [67]:
initial_val_frame

,Close,Open,High,Low,VOLUME,EMA_8,EMA_13,EMA_21,EMA_Signal,EMA_short,...,Lower_BB,ATR,TR,OBV,OBV_EMA20,OBV_Signal,VWAP_Signal,reddit_sentiment_log,tele_sentiment_log,news_sentiment_log
datetime,,,,,,,,,,,,,,,,,,,,,
2024-12-31 17:00:00,94120.94,94868.42,95047.71,93189.59,1332.40373,94617.102917,94342.895025,94052.159809,1,94394.046588,...,91330.978189,1174.988333,1858.12,-1332.40373,133.655000,0,-1,0.668578,1.968432,-6.382596
2024-12-31 18:00:00,93875.68,94120.94,94416.72,93779.80,816.81017,94452.342269,94276.150021,94036.116190,1,94314.297882,...,91418.780827,1160.111667,636.92,-816.81017,43.134507,0,-1,0.644116,1.968432,-6.413587
2024-12-31 19:00:00,94166.88,93875.68,94290.91,93712.45,462.79342,94388.906209,94260.540018,94048.003809,1,94291.618208,...,91518.783895,1001.461667,578.46,462.79342,83.102023,1,1,0.618698,2.029984,-6.449630
2024-12-31 20:00:00,93564.04,94166.88,94222.50,93450.17,733.04147,94205.602607,94161.040016,94004.007099,1,94179.683099,...,91657.553926,983.908333,772.33,-733.04147,5.374071,-1,-1,0.592473,2.029984,-6.482636
2024-12-31 21:00:00,93899.68,93564.04,93964.15,93504.67,337.52715,94137.619805,94123.702871,93994.522817,1,94136.605699,...,91830.596274,881.571667,459.48,337.52715,37.007698,1,-1,0.565590,2.029984,-6.514657
2024-12-31 22:00:00,93488.84,93899.68,93899.68,93375.58,315.53272,93993.446515,94033.008175,93948.551652,0,94036.949438,...,91979.914579,804.901667,524.10,-315.53272,3.432420,-1,-1,0.538200,2.029984,-6.543260
2024-12-31 23:00:00,93576.00,93488.84,93756.00,93375.38,336.57995,93900.680623,93967.721293,93914.683320,-1,93966.034140,...,92141.877951,558.651667,380.62,336.57995,35.160756,1,-1,0.510451,2.029984,-6.566459


In [ ]:
e = Engine(initial_cash = 100_000,asset_type='cryptocurrencies')
e.add_data(validate_frame)
e.add_strategy(XGBoostStrategy(loaded_model,initial_data= initial_val_frame)) # Pass both df_filtered and strategy_dict
stats = e.run()

 85%|████████▌ | 1734/2029 [00:03<00:00, 486.82it/s]

2025-03-11 03:00:00 Not enough cash to buy additional size.
2025-03-11 04:00:00 Not enough cash to buy additional size.
2025-03-11 05:00:00 Not enough cash to buy additional size.
2025-03-11 08:00:00 Not enough cash to buy additional size.
2025-03-11 09:00:00 Not enough cash to buy additional size.
2025-03-11 10:00:00 Not enough cash to buy additional size.
2025-03-11 12:00:00 Not enough cash to buy additional size.
2025-03-11 13:00:00 Not enough cash to buy additional size.
2025-03-11 14:00:00 Not enough cash to buy additional size.
2025-03-11 15:00:00 Not enough cash to buy additional size.
2025-03-11 17:00:00 Not enough cash to buy additional size.
2025-03-11 18:00:00 Not enough cash to buy additional size.
2025-03-11 23:00:00 Not enough cash to buy additional size.
2025-03-12 00:00:00 Not enough cash to buy additional size.
2025-03-12 01:00:00 Not enough cash to buy additional size.
2025-03-12 02:00:00 Not enough cash to buy additional size.
2025-03-12 03:00:00 Not enough cash to b

 92%|█████████▏| 1876/2029 [00:04<00:00, 450.82it/s]

2025-03-18 04:00:00 Not enough cash to buy additional size.
2025-03-18 05:00:00 Not enough cash to buy additional size.
2025-03-18 06:00:00 Not enough cash to buy additional size.
2025-03-18 07:00:00 Not enough cash to buy additional size.
2025-03-18 08:00:00 Not enough cash to buy additional size.
2025-03-18 09:00:00 Not enough cash to buy additional size.
2025-03-18 10:00:00 Not enough cash to buy additional size.
2025-03-18 11:00:00 Not enough cash to buy additional size.
2025-03-18 12:00:00 Not enough cash to buy additional size.
2025-03-18 13:00:00 Not enough cash to buy additional size.
2025-03-18 14:00:00 Not enough cash to buy additional size.
2025-03-18 15:00:00 Not enough cash to buy additional size.
2025-03-18 16:00:00 Not enough cash to buy additional size.
2025-03-18 17:00:00 Not enough cash to buy additional size.
2025-03-18 18:00:00 Not enough cash to buy additional size.
2025-03-18 19:00:00 Not enough cash to buy additional size.
2025-03-18 21:00:00 Not enough cash to b

100%|██████████| 2029/2029 [00:04<00:00, 440.73it/s]

Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |     151.47
Buy-and-Hold Total Return (%)            |      -6.25
Average Exposure to Asset (%)            |    -173.74
Strategy CAGR (%)                        |    5397.79
Buy & Hold CAGR (%)                      |     -26.97
Strategy Volatility (%)                  |      23.67
Buy & Hold Volatility (%)                |      11.66
Strategy Sharpe Ratio                    |     228.06
Buy & Hold Sharpe Ratio                  |      -2.31
Strategy Max Drawdown (%)                |     -21.78
Buy & Hold Max Drawdown (%)              |     -28.31
Number of Trades                         |    1611.00
Number of Buys                           |     771.00
Number of Sells                          |     840.00


In [70]:
e = Engine(initial_cash = 100_000,asset_type='cryptocurrencies')
e.add_data(test_frame)
e.add_strategy(XGBoostStrategy(loaded_model,initial_data= initial_test_frame)) # Pass both df_filtered and strategy_dict
stats = e.run()

 42%|████▏     | 256/613 [00:00<00:00, 483.74it/s]

2025-03-09 07:00:00 Not enough cash to buy additional size.
2025-03-09 09:00:00 Not enough cash to buy additional size.
2025-03-09 10:00:00 Not enough cash to buy additional size.
2025-03-09 11:00:00 Not enough cash to buy additional size.
2025-03-09 12:00:00 Not enough cash to buy additional size.
2025-03-09 16:00:00 Not enough cash to buy additional size.
2025-03-09 17:00:00 Not enough cash to buy additional size.
2025-03-09 18:00:00 Not enough cash to buy additional size.
2025-03-09 19:00:00 Not enough cash to buy additional size.
2025-03-09 20:00:00 Not enough cash to buy additional size.
2025-03-09 21:00:00 Not enough cash to buy additional size.
2025-03-09 22:00:00 Not enough cash to buy additional size.
2025-03-09 23:00:00 Not enough cash to buy additional size.
2025-03-10 00:00:00 Not enough cash to buy additional size.
2025-03-10 01:00:00 Not enough cash to buy additional size.
2025-03-10 05:00:00 Not enough cash to buy additional size.
2025-03-10 06:00:00 Not enough cash to b

 57%|█████▋    | 352/613 [00:00<00:00, 434.94it/s]

2025-03-12 22:00:00 Not enough cash to buy additional size.
2025-03-12 23:00:00 Not enough cash to buy additional size.
2025-03-13 00:00:00 Not enough cash to buy additional size.
2025-03-13 01:00:00 Not enough cash to buy additional size.
2025-03-13 03:00:00 Not enough cash to buy additional size.
2025-03-13 04:00:00 Not enough cash to buy additional size.
2025-03-13 05:00:00 Not enough cash to buy additional size.
2025-03-13 06:00:00 Not enough cash to buy additional size.
2025-03-13 07:00:00 Not enough cash to buy additional size.
2025-03-13 08:00:00 Not enough cash to buy additional size.
2025-03-13 09:00:00 Not enough cash to buy additional size.
2025-03-13 11:00:00 Not enough cash to buy additional size.
2025-03-13 12:00:00 Not enough cash to buy additional size.
2025-03-13 13:00:00 Not enough cash to buy additional size.
2025-03-13 14:00:00 Not enough cash to buy additional size.
2025-03-13 15:00:00 Not enough cash to buy additional size.
2025-03-13 16:00:00 Not enough cash to b

 79%|███████▉  | 483/613 [00:01<00:00, 418.29it/s]

2025-03-18 04:00:00 Not enough cash to buy additional size.
2025-03-18 05:00:00 Not enough cash to buy additional size.
2025-03-18 06:00:00 Not enough cash to buy additional size.
2025-03-18 07:00:00 Not enough cash to buy additional size.
2025-03-18 08:00:00 Not enough cash to buy additional size.
2025-03-18 09:00:00 Not enough cash to buy additional size.
2025-03-18 10:00:00 Not enough cash to buy additional size.
2025-03-18 11:00:00 Not enough cash to buy additional size.
2025-03-18 12:00:00 Not enough cash to buy additional size.
2025-03-18 13:00:00 Not enough cash to buy additional size.
2025-03-18 14:00:00 Not enough cash to buy additional size.
2025-03-18 15:00:00 Not enough cash to buy additional size.
2025-03-18 16:00:00 Not enough cash to buy additional size.
2025-03-18 17:00:00 Not enough cash to buy additional size.
2025-03-18 18:00:00 Not enough cash to buy additional size.
2025-03-18 19:00:00 Not enough cash to buy additional size.
2025-03-18 21:00:00 Not enough cash to b

100%|██████████| 613/613 [00:01<00:00, 445.64it/s]

Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |       7.75
Buy-and-Hold Total Return (%)            |       4.00
Average Exposure to Asset (%)            |     -44.99
Strategy CAGR (%)                        |     197.23
Buy & Hold CAGR (%)                      |      95.99
Strategy Volatility (%)                  |      18.38
Buy & Hold Volatility (%)                |      13.81
Strategy Sharpe Ratio                    |      10.73
Buy & Hold Sharpe Ratio                  |       6.95
Strategy Max Drawdown (%)                |     -17.71
Buy & Hold Max Drawdown (%)              |     -17.68
Number of Trades                         |     402.00
Number of Buys                           |     187.00
Number of Sells                          |     215.00


In [71]:
fig = e.plot()

AttributeError: 'NoneType' object has no attribute 'index'